In [32]:
from langchain_groq import ChatGroq

In [33]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

APIConnectionError: Connection error.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/116221099686929094-account-executive-midmarket-sales-google-customer-solutions")
page_data = loader.load().pop().page_content
print(page_data)

Account Executive, Mid-Market Sales, Google Customer Solutions — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results1,810  jobs matchedAccount Executive, Mid-Market Sales, Google Customer SolutionsMumbai, Maharashtra, India; Gurgaon, Haryana, India; +2 more; +1 moreSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaSenior Software Engineer, AI/ML GenAI, Google Cloud AISunnyvale, CA, USA; Kirkland, WA, USASenior Program Manager, Supply Chain Operations, Hardware FulfillmentSunnyvale, CA, USASoftware Engineer, Silicon Engineering ProductivityBengaluru, Karnataka, IndiaData Center Security ManagerSkien, NorwaySenior Software Engineer, Search UI and AI InfrastructureSão Paulo, State of São Paulo

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res[0]

{'role': 'Account Executive, Mid-Market Sales, Google Customer Solutions',
 'experience': 'Mid',
 'skills': ['Knowledge of traditional and online marketing strategy and tactics',
  'Knowledge of the local market with excellent analytical skills',
  'Ability to think strategically about complex issues and develop recommendations and action plans',
  'Demonstrated ability to lead and manage multiple projects with a broad range of internal and external constituents simultaneously'],
 'description': 'Businesses that partner with Google come in all shapes, sizes and market caps, and no one Google advertising solution works for all. Your knowledge of online media combined with your communication skills and analytical abilities shapes how new and existing businesses grow.'}

In [23]:
type(json_res[0])

dict

In [24]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [28]:
job = json_res[0]
job['skills']

['Knowledge of traditional and online marketing strategy and tactics',
 'Knowledge of the local market with excellent analytical skills',
 'Ability to think strategically about complex issues and develop recommendations and action plans',
 'Demonstrated ability to lead and manage multiple projects with a broad range of internal and external constituents simultaneously']

In [29]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [30]:
job

{'role': 'Account Executive, Mid-Market Sales, Google Customer Solutions',
 'experience': 'Mid',
 'skills': ['Knowledge of traditional and online marketing strategy and tactics',
  'Knowledge of the local market with excellent analytical skills',
  'Ability to think strategically about complex issues and develop recommendations and action plans',
  'Demonstrated ability to lead and manage multiple projects with a broad range of internal and external constituents simultaneously'],
 'description': 'Businesses that partner with Google come in all shapes, sizes and market caps, and no one Google advertising solution works for all. Your knowledge of online media combined with your communication skills and analytical abilities shapes how new and existing businesses grow.'}

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at Atliq. Atliq is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABC Company 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at Atliq. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Google Customer Solutions with AtliQ's Expertise

Dear Hiring Manager,

I came across the Account Executive, Mid-Market Sales, Google Customer Solutions role at Google, and I was impressed by the company's commitment to providing tailored advertising solutions to businesses of all sizes. As a Business Development Executive at AtliQ, I believe our company's capabilities in AI and software consulting can complement Google's efforts in empowering businesses to grow.

At AtliQ, we have a proven track record of delivering customized solutions that drive scalability, process optimization, cost reduction, and increased efficiency. Our expertise in developing automated tools can help Google's mid-market sales team streamline their operations, enhance customer engagement, and ultimately drive revenue growth. With our analytical skills and knowledge of online media, we can help develop strategic recommendations and action plans to tackle complex issues.

Our portfolio showcase